In [8]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [9]:

import zipfile

# Replace 'DATASET.zip' with the actual name of your zip file
with zipfile.ZipFile('archive (5).zip', 'r') as zip_ref:     # replace it with ur zipfilename
    zip_ref.extractall('DATASET') # extracts to a directory called 'DATASET'

In [10]:
# Paths to dataset folders
TRAIN_DIR = '/content/DATASET/train'
TEST_DIR = '/content/DATASET/test'
VALID_DIR = '/content/DATASET/valid'

# Image size and batch size
IMG_SIZE = (128, 128)  # Resize all images to 128x128
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
valid_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'   # categorical
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  #categorical
)

valid_generator = valid_datagen.flow_from_directory(
    VALID_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  #categorical
)




Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [11]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # softmax for multi classification
    ])

    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',  #sparse_categorical_crossentropy
                  metrics=['accuracy'])
    return model


model = build_model()
model.summary()

#Training the Model


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=valid_generator
)



Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


946/946 ━━━━━━━━━━━━━━━━━━━━ 70s 71ms/step - accuracy: 0.8537 - loss: 0.3284 - val_accuracy: 0.9397 - val_loss: 0.1607
Epoch 2/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 65s 68ms/step - accuracy: 0.9319 - loss: 0.1832 - val_accuracy: 0.9514 - val_loss: 0.1390
Epoch 3/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 81s 67ms/step - accuracy: 0.9427 - loss: 0.1575 - val_accuracy: 0.9570 - val_loss: 0.1118
Epoch 4/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 66s 69ms/step - accuracy: 0.9483 - loss: 0.1400 - val_accuracy: 0.9563 - val_loss: 0.1118
Epoch 5/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 64s 67ms/step - accuracy: 0.9545 - loss: 0.1307 - val_accuracy: 0.9589 - val_loss: 0.1053


In [13]:
# Saving model
model.save('wildfire.h5')   #potato_disease.h5

In [14]:
# 4. Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.9702 - loss: 0.0833
Test Accuracy: 0.966349184513092


In [15]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')

# Load the trained model
model = load_model('/content/wildfire.h5')

# Path to your custom image
image_path = r"/content/-60.6867,50.26079.jpg" # Replace with your image path

# Preprocess the image
IMG_SIZE = (128, 128)  # Must match the size used during training
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(128,128))  # Load and resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Rescale to 0-1 range
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(image)
print("prediction prob" , prediction)
# Interpretation
if prediction[0][0] < 0.5:
    print(f"Wildfire detected")
else:
    print(f"wildfire not detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
prediction prob [[0.46138567]]
Wildfire detected


In [ ]:
# below code is for multiclassification prediction do not execute it

In [17]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')

# Load the trained model
model = load_model('/content/wildfire.h5')  # Updated model path

# Path to your custom image
image_path = r"/content/-60.6867,50.26079.jpg"  # Replace with your image path

# Preprocess the image
IMG_SIZE = (128, 128)  # Must match the size used during training

def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)  # Load and resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Rescale to 0-1 range
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(image)

# Class labels (ensure they match the model's training)
class_labels = ['wildfire detected','not detected']

# Get the predicted class index
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_label = class_labels[predicted_class_index]

# Output the prediction
print(f"The image is predicted to be: {predicted_class_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
The image is predicted to be: wildfire detected
